In [1]:
import pandas as pd
import numpy as np

In [2]:
movies =pd.read_csv('TMDB_MOVIE_DATASET/tmdb_5000_movies.csv')
credits = pd.read_csv('TMDB_MOVIE_DATASET/tmdb_5000_credits.csv')

In [3]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [5]:
#credits['cast'][0]
#credits['crew'][0]

In [6]:
movies.shape

(4803, 20)

In [7]:
credits.shape

(4803, 4)

In [8]:
# now we we merge them to create a single table
#movies.merge(credits,on='title').shape
movies = movies.merge(credits,on='title')

In [9]:
# now we have to perform analysis on various columns to identify which colums are important in the creeation of our tags
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [10]:
#we need to drop a few columns so that we can keep precise data for our movie recommendation system
#only considering catgorical values for now because i am using content based recommender system
#genres
#id
#keywords
#title
#overview
#cast
#crew
movies=movies[['genres','movie_id','keywords','title','overview','cast','crew']]
movies.head(1)

,genres,movie_id,keywords,title,overview,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [11]:
# for a recommender system we only need 3 columns like id, title and tags
# To create tags we need to merge and pre process some data like 
#[genre,keywords,overview,cast,crew] ---> [tags]
#first we gonna change the format of the data
# from cast we will choose only top 5 actors 
# also we only need director name from the crew column

In [12]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genres    4809 non-null   object
 1   movie_id  4809 non-null   int64 
 2   keywords  4809 non-null   object
 3   title     4809 non-null   object
 4   overview  4806 non-null   object
 5   cast      4809 non-null   object
 6   crew      4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 300.6+ KB


In [13]:
#checking for null values and since the null values are very less we can drop it.
movies.isna().sum()

genres      0
movie_id    0
keywords    0
title       0
overview    3
cast        0
crew        0
dtype: int64

In [14]:
movies.dropna(inplace=True)

In [15]:
movies.isna().sum()

genres      0
movie_id    0
keywords    0
title       0
overview    0
cast        0
crew        0
dtype: int64

In [16]:
#checking for duplicate values and since no duplicate values we wont do anything.
movies.duplicated().sum()

0

In [17]:
# Now the data is in wierd format so we need to convert it into proper format before we can use it for eg.
movies.genres.iloc[0]
#'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]

# convert this into
#[Action, Adventure,Fantasy,Science Fiction...etc] format

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [18]:
# For this we will create a helper function

import ast

def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L

In [19]:
movies['genres']= movies['genres'].apply(convert)

In [20]:
movies['keywords'] = movies['keywords'].apply(convert)

In [21]:
movies.head(1)

,genres,movie_id,keywords,title,overview,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [22]:
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L

In [23]:
movies['cast'] = movies['cast'].apply(convert)
movies.head(1)

,genres,movie_id,keywords,title,overview,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [24]:
movies['cast'][0]

['Sam Worthington',
 'Zoe Saldana',
 'Sigourney Weaver',
 'Stephen Lang',
 'Michelle Rodriguez',
 'Giovanni Ribisi',
 'Joel David Moore',
 'CCH Pounder',
 'Wes Studi',
 'Laz Alonso',
 'Dileep Rao',
 'Matt Gerald',
 'Sean Anthony Moran',
 'Jason Whyte',
 'Scott Lawrence',
 'Kelly Kilgour',
 'James Patrick Pitt',
 'Sean Patrick Murphy',
 'Peter Dillon',
 'Kevin Dorman',
 'Kelson Henderson',
 'David Van Horn',
 'Jacob Tomuri',
 'Michael Blain-Rozgay',
 'Jon Curry',
 'Luke Hawker',
 'Woody Schultz',
 'Peter Mensah',
 'Sonia Yee',
 'Jahnel Curfman',
 'Ilram Choi',
 'Kyla Warren',
 'Lisa Roumain',
 'Debra Wilson',
 'Chris Mala',
 'Taylor Kibby',
 'Jodie Landau',
 'Julie Lamm',
 'Cullen B. Madden',
 'Joseph Brady Madden',
 'Frankie Torres',
 'Austin Wilson',
 'Sara Wilson',
 'Tamica Washington-Miller',
 'Lucy Briant',
 'Nathan Meister',
 'Gerry Blair',
 'Matthew Chamberlain',
 'Paul Yates',
 'Wray Wilson',
 'James Gaylyn',
 'Melvin Leno Clark III',
 'Carvon Futrell',
 'Brandon Jelkes',
 'Mica

In [25]:
movies['cast'] = movies['cast'].apply(lambda x:x[0:3])

In [26]:
movies.head()

,genres,movie_id,keywords,title,overview,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",Spectre,A cryptic message from Bond’s past sends him o...,"[Daniel Craig, Christoph Waltz, Léa Seydoux]","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",The Dark Knight Rises,Following the death of District Attorney Harve...,"[Christian Bale, Michael Caine, Gary Oldman]","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...",John Carter,"John Carter is a war-weary, former military ca...","[Taylor Kitsch, Lynn Collins, Samantha Morton]","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [27]:
#this is useed to fetch th director name from the crew
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

movies['crew'] = movies['crew'].apply(fetch_director)

In [28]:
#for 5 random samples
movies.sample(5)

,genres,movie_id,keywords,title,overview,cast,crew
859,"[Action, Adventure, Comedy, Family, Fantasy, S...",14623,"[secret organization, based on tv series, gold...",Thunderbirds,Dangerous missions are the bread and butter of...,"[Brady Corbet, Soren Fulton, Debora Weston]",[Jonathan Frakes]
3439,"[Romance, Drama]",18923,"[gay, aids, based on novel, art house, tantrum...",A Home at the End of the World,"Three friends form a bond over the year, Johna...","[Colin Farrell, Dallas Roberts, Robin Wright]",[Michael Mayer]
223,"[Action, Science Fiction]",2789,"[prison, dystopia, matter of life and death, o...",The Chronicles of Riddick,After years of outrunning ruthless bounty hunt...,"[Vin Diesel, Thandie Newton, Karl Urban]",[David Twohy]
1961,"[Crime, Action, Thriller]",290864,"[martial arts, kung fu, serial killer]",Kung Fu Jungle,A martial arts instructor working at a police ...,"[Donnie Yen, Charlie Yeung, Wang Baoqiang]",[Teddy Chan]
3216,"[Drama, Music, Romance]",25196,"[taxi, country music, journalist, guitar, bar,...",Crazy Heart,When reporter Jean Craddock interviews Bad Bla...,"[Jeff Bridges, Maggie Gyllenhaal, Colin Farrell]",[Scott Cooper]


In [29]:
#This is used t remove the sapce in between the tags so that they can act as a whole, eg. ryan and Reynolds will be two tags but
#RyanReynolds will be one complete tag

def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [30]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [31]:
movies.head()

,genres,movie_id,keywords,title,overview,cast,crew
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,"[Adventure, Fantasy, Action]",285,"[ocean, drugabuse, exoticisland, eastindiatrad...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,"[Action, Adventure, Crime]",206647,"[spy, basedonnovel, secretagent, sequel, mi6, ...",Spectre,A cryptic message from Bond’s past sends him o...,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,"[Action, Crime, Drama, Thriller]",49026,"[dccomics, crimefighter, terrorist, secretiden...",The Dark Knight Rises,Following the death of District Attorney Harve...,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,"[Action, Adventure, ScienceFiction]",49529,"[basedonnovel, mars, medallion, spacetravel, p...",John Carter,"John Carter is a war-weary, former military ca...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [32]:
# now we want to convert our overview
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies.head(1)

,genres,movie_id,keywords,title,overview,cast,crew
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]


In [33]:
movies['tags'] =  movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew'] + movies['overview']
movies.head(1)

,genres,movie_id,keywords,title,overview,cast,crew,tags
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction, c..."


In [34]:
new_df = movies[['movie_id','title','tags']]
new_df.head(1)

,movie_id,title,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, c..."


In [35]:
#convert this list into string
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

<ipython-input-35-876a72c679ec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [36]:
new_df['tags'][0]

'Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [37]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())
new_df.head(1)

<ipython-input-37-092fda7fd74f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


,movie_id,title,tags
0,19995,Avatar,action adventure fantasy sciencefiction cultur...


In [38]:
#Now there is a problem when we look at the list coz it contains similar words 'accept', 'acceptance', 'accepted', 'accepts'
# and for that we'll use a NLP library i.e. nltk
!pip install nltk

In [39]:
import nltk

In [40]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [41]:
#create a stemmer function to so that we can convert all the tags into their root words

In [42]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)


In [43]:
new_df['tags'] = new_df['tags'].apply(stem)
new_df['tags'][0]

<ipython-input-43-8a9fc58609d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


'action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization.'

In [44]:
#convert tags into vectors and now we can recommend him the movies whose vectors are closest to his original movie vectors
#we need to convert text into vectors and this process is called text vectorization
#There are various techniques to it but the one that we'll use is 'Bag of words' others are eg,tfif,word2vec etc
# tags = tag 1+ tag 2 .....tag 4806 ---> we will use 5000 most common words

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10000,stop_words='english')

In [46]:
#By default it returns value in sparse matrix so we onvert it into numpy array
vector = cv.fit_transform(new_df['tags']).toarray()

In [50]:
#vector.shape

(4806, 10000)

In [51]:
#cv.get_feature_names()

['00',
 '000',
 '007',
 '10',
 '100',
 '10th',
 '11',
 '12',
 '12th',
 '13',
 '14',
 '15',
 '150',
 '15th',
 '16',
 '16th',
 '17',
 '17th',
 '18',
 '1863',
 '1890',
 '18th',
 '18thcenturi',
 '19',
 '1910',
 '1920',
 '1927',
 '1930',
 '1930s',
 '1937',
 '1940',
 '1940s',
 '1941',
 '1944',
 '1945',
 '1950',
 '1950s',
 '1955',
 '1959',
 '1960',
 '1960s',
 '1962',
 '1964',
 '1965',
 '1967',
 '1969',
 '1970',
 '1970s',
 '1971',
 '1972',
 '1973',
 '1974',
 '1976',
 '1977',
 '1979',
 '1980',
 '1980s',
 '1984',
 '1985',
 '1986',
 '1987',
 '1990',
 '1994',
 '1995',
 '1996',
 '1997',
 '1999',
 '19th',
 '19thcenturi',
 '20',
 '200',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2007',
 '2008',
 '2009',
 '2011',
 '2012',
 '20th',
 '21st',
 '21stcenturi',
 '22nd',
 '23',
 '24',
 '25',
 '27',
 '28',
 '29',
 '30',
 '300',
 '35',
 '3d',
 '40',
 '400',
 '47',
 '50',
 '500',
 '51',
 '60',
 '60s',
 '70',
 '7th',
 '80',
 'aaron',
 'aaroneckhart',
 'aaronseltz',
 'aarontaylor',
 'abandon',
 'abbi',
 'abbi

In [52]:
# now we have to calculate distance of each vector from others
# increase in distance will decrease in similarity i.e. inversely proportional
# in this we cannot calculate eucliden distace so we'll calculate cosine distance i.e. thetha between them

In [53]:
from sklearn.metrics.pairwise import cosine_similarity

In [54]:
similarity = cosine_similarity(vector)

In [57]:
similarity.shape

(4806, 4806)

In [56]:
similarity

array([[1.        , 0.07194017, 0.07396705, ..., 0.03933686, 0.        ,
        0.        ],
       [0.07194017, 1.        , 0.055577  , ..., 0.02216755, 0.        ,
        0.02423931],
       [0.07396705, 0.055577  , 1.        , ..., 0.02279212, 0.        ,
        0.        ],
       ...,
       [0.03933686, 0.02216755, 0.02279212, ..., 1.        , 0.03739788,
        0.03976214],
       [0.        , 0.        , 0.        , ..., 0.03739788, 1.        ,
        0.08178608],
       [0.        , 0.02423931, 0.        , ..., 0.03976214, 0.08178608,
        1.        ]])

In [58]:
#create a helper function to  get title then sort them in desc order and to hold first value we use enumerate
# and also we use lambda funnction so that we can pick our recommedation from 2nd position

def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)

In [59]:
 #converting it into pickle file
import pickle

In [61]:
pickle.dump(new_df,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))